In [1]:
# Notebook to compute, partition, and invert quasigeostrophic potential vorticity
# For MEA 717, spring 2022, Gary Lackmann
# Kudos to Michael Gray for contributing a much faster method of computing Laplacians
#
# This code computes the QG PV vorticity, partitions it, and inverts specified pieces
#
# Some of the following imports are not used right now, but will retain for future flexibility
import os
import subprocess
import matplotlib.pyplot as plt
from matplotlib.cm import get_cmap
import matplotlib.cbook as cbook
from matplotlib.colors import Normalize
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation

import pandas as pd
import xarray as xr
import numpy as np
import math
from numpy import *
from pylab import *
import pygrib
import pyproj

from siphon.catalog import TDSCatalog
from siphon.http_util import session_manager
from datetime import datetime, timedelta
from xarray.backends import NetCDF4DataStore
from netCDF4 import Dataset
import metpy as metpy
import metpy.calc as mpcalc
from metpy.plots import ctables
from metpy.units import units
from metpy.plots import add_metpy_logo, add_timestamp
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import scipy.ndimage as ndimage
from scipy.ndimage import gaussian_filter

import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
from cartopy import config
import wrf
from wrf import (to_np, interplevel, geo_bounds, getvar, smooth2d, get_cartopy, cartopy_xlim,
                 cartopy_ylim, latlon_coords)
# Download and add the states and coastlines
states = NaturalEarthFeature(category="cultural", scale="50m",
                          facecolor="none", name="admin_1_states_provinces_shp")
import glob

In [2]:
# SPECIFY Data File

# Set up for saving graphics files, create directory for plots, if needed
plotsdir = '/home/jupyter-sdsmit12@ncsu.edu/plots/'
if os.path.isdir(plotsdir) != 1:
    subprocess.call(["mkdir","-p",plotsdir])
os.chdir(plotsdir)

# Set directory where wrfout files reside, and list the files for processing.  Set up for a directory with only wrfout files.
#datafiles = (glob.glob("/scratch/morgan/forAmanda/nasaterrian/wrfout_d01_1996-09-07*"))
datafiles = (glob.glob("/scratch/sawyer/wrfout_files/wrfout_d01_2008-12-18_12:00:00"))
datafiles.sort()
numfiles=len(datafiles)
print(numfiles)
print(datafiles[0])

# Set up loop to run same plots for multiple different times (assume here that we have 1 time per wrfout file)
#for j in range(0,numfiles):

sfc_p=[]
pwater=[]

for j in range(0,numfiles):
    ncfile = Dataset(datafiles[j])
    Time=wrf.extract_times(ncfile, timeidx=0, method='cat', squeeze=True, cache=None, meta=False, do_xtime=False)
    timestr=(str(Time))
    # Set up one time string for plot titles, another for file names
    titletime=(timestr[0:10]+' '+timestr[11:16])
    filetime=(timestr[0:10]+'_'+timestr[11:13])
    print('WRF valid time: ',filetime)
    #print(ncfile)
    # Get all the variables we need
    z = getvar(ncfile, "z")
    #dbz3 =getvar(ncfile, "dbz")
    p = getvar(ncfile, "pressure")
    #slp = getvar(ncfile, "slp")
    #rainc = getvar(ncfile, "RAINC")
    #rainnc = getvar(ncfile, "RAINNC")
   # mdbz = getvar(ncfile, "mdbz")
    #tk = getvar(ncfile, "tk")
    ua = getvar(ncfile, "ua", units="kt")
    va = getvar(ncfile, "va", units="kt")
    pvo = getvar(ncfile, "pvo")
    wspd = getvar(ncfile, "wspd_wdir", units="kts")[0,:]
    pwat = getvar(ncfile, "pw")
    raintot = (rainc + rainnc)*0.0393701 #in inches

    pwater.append(pwat)
    sfc_p.append(slp)
    print(np.max(pwater),np.min(pwater))


    # Do vertical interpolation to needed pressure surfaces - could do a loop over vertical levels at some point
    ht_850 = interplevel(z, p, 850.)
    ht_500 = interplevel(z, p, 500.)
    ht_250 = interplevel(z, p, 250.)
    #dbz_850 = interplevel(dbz3, p, 850.)
    u_850 = interplevel(ua, p, 850)
    v_850 = interplevel(va, p, 850)
    u_500 = interplevel(ua, p, 500)
    v_500 = interplevel(va, p, 500)
    u_250 = interplevel(ua, p, 250)
    v_250 = interplevel(va, p, 250)
    wspd_500 = interplevel(wspd, p, 500)
    wspd_250 = interplevel(wspd, p, 250)
    tk_850 = interplevel(tk, p, 850)
    PV_850 = interplevel(pvo, p, 850)


    # Download and add the states and coastlines
    states = NaturalEarthFeature(category="cultural", scale="50m",facecolor="none", name="admin_1_states_provinces_shp")
    # Get the latitude and longitude points
    lats, lons = latlon_coords(slp)
    # Smooth the sea level pressure since it tends to be noisy near complex terrain
    smooth_slp = smooth2d(slp, 3, cenweight=4)

    sfc_p.append(smooth_slp)
    print(np.shape(smooth_slp))

    # Make reflectivity below 0 or -5 NaN to avoid plotting, get reflectivity color table
    #mdbz_plot = np.where(mdbz > 0., mdbz, "NaN")
    #ctables.registry.get_colortable('NWSReflectivity')

    plotproj = crs.LambertConformal(central_longitude=200., central_latitude=20.,standard_parallels=[20, 50])
    #plotproj = ccrs.PlateCarree(central_longitude = 200)
    fig = plt.figure(figsize=(10,8))
    ax=plt.subplot(111, projection=plotproj)
    ax.set_extent([185, 250, 10, 60],crs.PlateCarree())

    ax.add_feature(cfeature.STATES.with_scale('50m'), linewidth=0.25, edgecolor='white')
    ax.coastlines('50m', linewidth=0.4, color='white')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
 
    # Add height contours with shaded PV 
    rain_levels = np.arange(1, 25, 0.5)
    pmsl_levels = np.arange(900, 1060, 4)
    dbz_levels = np.arange(0, 60, 4)
    pwat_levels = np.arange(0, 65.1, 1)
    contours = plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp), levels=pmsl_levels, colors="black", transform=crs.PlateCarree())
    plt.clabel(contours, inline=1, fontsize=10, fmt="%i") 
    #
    pw_contours = plt.contourf(to_np(lons), to_np(lats), to_np(pwat),
                             cmap=get_cmap("viridis"), levels=pwat_levels, alpha=1,transform=crs.PlateCarree())
    cbar = plt.colorbar(pw_contours, ax=ax, orientation="horizontal", pad=.03, shrink=.8, aspect=50)
    cbar.set_label("Precipitable Water (mm)", fontsize = 16)
    #rain_contours = plt.contourf(to_np(lons), to_np(lats), to_np(raintot), levels=rain_levels,
    #                         cmap=get_cmap("rainbow"),norm=Normalize(0,25), vmin=0, vmax=25, alpha=.5,transform=crs.PlateCarree())
    #plt.colorbar(rain_contours, ax=ax, orientation="horizontal", pad=.03, shrink=.8, aspect=50)

    plt.title(" SLP (hPa) and Precipitable Water (mm)")
    # Create separate plot file and save as .png, then show and close
    outTPlotName= 'dbz.png'
    plt.savefig(plotsdir+'/'+outTPlotName)
    plt.show()

    plt.savefig('output_' + filetime + '.png')
    plt.close(fig)
    # Loop over the data

for i in range(numfiles):
    fig, ax = plt.subplots()
    ax.plot(numfiles[i])
    # Generate a unique filename
    filename = f'figure_{i}.png'
    # Provide the full path to save the figure in the figures directory
    fig.savefig(os.path.join(plotsdir+'/'+filename))
    plt.close(fig)



    plt.close()


   

0


IndexError: list index out of range

In [ ]:
'''
slp_array = np.array(ivan_slp)
print(np.shape(slp_array))
print(np.min(slp_array))
#print(np.values(slp_array[40]))
slp_array[:][slp_array[:]>0].min()
indices = np.where(slp_array == 955.89075)
print(indices)
print(np.min(slp_array[27])) #minimum pressure at landfall

#slp_values = slp_array[:, 0] #extract the values of slp_array from the first column for all rows (i.e., all time indices).
#print(slp_values)
'''

In [ ]:
#make animation for all time indices

# Create a list of figures
figs = []

# Loop over each file and create a plot for each one
for j in range(0,numfiles):
    # ...
    fig = plt.figure(figsize=(16,12))
    # ...
    figs.append(fig)

# Create the animation
#ani = animation.ArtistAnimation(figs, list, interval=1000, blit=True)

anim = FuncAnimation(figs, callable, frames=len(smooth_slp), interval=100)

ax.set_title('Precip)
# Save the animation to a file
anim.save('animation.mp4')

# Create separate plot file and save as .png, then show and close
#outTPlotName= '.png'
#plt.savefig(plotsdir+'/'+outTPlotName)
#plt.show()
plt.close()

In [ ]:
# Smooth the sea level pressure since it tends to be noisy near complex terrain
# The smoother can be adjusted, experiment with different values
smooth_slp = smooth2d(slp, 3, cenweight=4)

# Make reflectivity below 0 or -5 NaN to avoid plotting, get reflectivity color table
mdbz_plot = np.where(mdbz > 0., mdbz, "NaN")
ctables.registry.get_colortable('NWSReflectivity')
        
# Now do the plotting, first SLP and reflectivity
# Create figure, adjust size for higher quality or resolution
fig = plt.figure(figsize=(16,12))

# Set the GeoAxes to the projection used by WRF, add state boundaries and coastlines
ax = plt.axes(projection=plot_proj)
ax.add_feature(states, linewidth=.5, edgecolor="brown")
ax.coastlines('50m', linewidth=0.8)

# Make the contour outlines and filled contours for reflectivity.
plt.contourf(to_np(lons), to_np(lats), to_np(mdbz_plot), 10, transform=crs.PlateCarree(),
        cmap=ctables.registry.get_colortable('NWSReflectivity'), norm=Normalize(0,60), vmin=0, vmax=60, alpha=.5)

# Plot SLP with contour labels
cs = plt.contour(to_np(lons), to_np(lats), to_np(smooth_slp), levels=range(936,1068,4), colors='black', 
                linestyles='solid', transform=crs.PlateCarree())
plt.clabel(cs, fmt= '%.0f', inline = True)

# Add color bar
m = plt.cm.ScalarMappable(cmap=ctables.registry.get_colortable('NWSReflectivity'))
m.set_array(mdbz)
m.set_clim(0., 60.)
plt.colorbar(m, ax=ax, shrink=.75, boundaries=np.linspace(0, 60, 13), alpha=.5)

# Set the map bounds
ax.set_xlim(cartopy_xlim(smooth_slp))
ax.set_ylim(cartopy_ylim(smooth_slp))

# Add the lat/long gridlines (for Mercator projection)
ax.gridlines(color="white", linestyle="dotted")

# Set plot title
plt.title("Sea Level Pressure (hPa) and composite reflectivity,"+' '+titletime+' UTC')

# Create separate plot file and save, then show and close
outTPlotName= 'SLP_maxdBZ'+filetime+'.png'
#plt.savefig(plotsdir+outTPlotName)
# uncomment following line if you want plots to show inline, otherwise just check for files in ./plots
plt.show()
plt.close()